# # Treinamento do Modelo de Detecção de Boletos

### Imports

In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pickle
import shap
import os


In [ ]:
MODEL_PATH = 'modelo/modelo_boleto.pkl'

### # 2. Carregamento e Prepara os Dados

In [ ]:
try:
    with open('completos.json', 'r', encoding='utf-8') as f:
        dados = json.load(f)
except FileNotFoundError:
    print("Erro: Arquivo completos.json não encontrado.")
    exit(1)

df = pd.DataFrame(dados)
print('Dados carregados:')
print(df.head())


### # Mapear bancos para índices numéricos

In [ ]:
mapeamento_bancos = {
    'Banco do Brasil': 0,
    'Itaú': 1,
    'Bradesco': 2,
    'Santander': 3,
    'Caixa Econômica': 4,
    'Banco Digio S.A.': 5,
    'CM CAPITAL MARKETS CORRETORA DE CÂMBIO, TÍTULOS E VALORES MOBILIÁRIOS LTDA': 6,
    'Banco Clássico S.A.': 7,
    'Credialiança Cooperativa de Crédito Rural': 8,
    'CREDICOAMO CREDITO RURAL COOPERATIVA': 9,
    # Bancos falsos
    'OLIVEIRA TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIARIOS S.A.': 10,
    'Pagseguro Internet S.A. – PagBank': 11,
    'NU Pagamentos S.A. – Nubank': 12,
    'ATIVA INVESTIMENTOS S.A. CORRETORA DE TÍTULOS, CÂMBIO E VALORES': 13,
    'Banco Inbursa S.A.': 14,
    'SOROCRED CRÉDITO, FINANCIAMENTO E INVESTIMENTO S.A.': 15,
    'Banco Finaxis S.A.': 16,
    'SOCRED S.A. – SOCIEDADE DE CRÉDITO AO MICROEMPREENDEDOR E À EMPRESA DE PEQUENO P': 17
}
df['banco'] = df['banco'].map(mapeamento_bancos).fillna(0).astype(float)


# Mapear classe
df['classe'] = df['classe'].map({
    'caracteristicasVerdadeiras': 1,
    'caracteristicasFalsas': 0
}).astype(float)

NameError: name 'df' is not defined

### # Processar linha digitável e selecionar as features

In [ ]:
df['linhaDigitavel'] = df['linhaDigitavel'].str.replace(' ', '').str.replace('.', '').astype(str)
df['linha_codBanco'] = df['linhaDigitavel'].str.slice(0, 3).fillna('0').astype(float)
df['linha_moeda'] = df['linhaDigitavel'].str.slice(3, 4).fillna('9').astype(float)
df['linha_valor'] = df['linhaDigitavel'].str.slice(-10).fillna('0').astype(float)

# Selecionar features
features = ['banco', 'codigoBanco', 'agenciaBanco', 'valorDocumento', 'linha_codBanco', 'linha_moeda', 'linha_valor']
X = df[features].fillna(0).astype(float)
y = df['classe'].fillna(0).astype(float)

print('Features preparadas:')
print(X.head())

### Divisão dos dados

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Treino: {X_train.shape}, Teste: {X_test.shape}')

### Treinar o modelo

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


# Avaliar
y_pred = model.predict(X_test)
print('Acurácia:', accuracy_score(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred, target_names=['Falso', 'Verdadeiro']))


## Análise com SHAP - XAI

In [ ]:
# 5. Análise com SHAP
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

## # Plotar resumo para a classe 'Falso' (índice 0)

In [ ]:
shap.summary_plot(shap_values[0], X_test, plot_type='bar', feature_names=features, show=False)
plt.title('Importância das Features para Classe Falso')
plt.tight_layout()
plt.show()

### # Plotar resumo para a classe 'Verdadeiro' (índice 1)


In [ ]:
shap.summary_plot(shap_values[1], X_test, plot_type='bar', feature_names=features, show=False)
plt.title('Importância das Features para Classe Verdadeiro')
plt.tight_layout()
plt.show()

### Salvar o modelo

In [ ]:
pickle.dump(model, open(MODEL_PATH, 'wb'))
print(f'✅ Modelo salvo em: {MODEL_PATH}')

# 7. Teste com Exemplos
testes = [
    {  # Falso: OLIVEIRA TRUST
        'banco': 10,
        'codigoBanco': 700,
        'agenciaBanco': 712,
        'valorDocumento': 834629.43,
        'linha_codBanco': 111,
        'linha_moeda': 0,
        'linha_valor': 83462943
    },
    {  # Verdadeiro: Nubank
        'banco': 12,
        'codigoBanco': 188,
        'agenciaBanco': 101,
        'valorDocumento': 731538.0,
        'linha_codBanco': 260,
        'linha_moeda': 3,
        'linha_valor': 94228611
    }
]

for i, teste in enumerate(testes):
    dado_teste = pd.DataFrame([teste]).astype(float)
    pred = model.predict(dado_teste)[0]
    prob = model.predict_proba(dado_teste)[0].tolist()
    print(f'Boleto {i+1}: Predição = {"Verdadeiro" if pred == 1 else "Falso"}, Probabilidades = {prob}')
    
    # Explicações SHAP para o teste
    shap_values_teste = explainer.shap_values(dado_teste)
    shap.initjs()
    shap.force_plot(explainer.expected_value[0], shap_values_teste[0], dado_teste, feature_names=features)
```